In [1]:
"""
This notebook will be used as a template for each dataset giving the key outputs I need for presentations
and publications
"""

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw

import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
# Type in datasets you would like to combine.  Adjust totvids to reflect the total number of videos. Also type in the
# umppx and fps in the conversion dictionary.

logplot = 'confocal_625ms_40x_logplot_old'
Mplot = 'confocal_625ms_40x_Mplot_old'
Dplot = 'confocal_625ms_40x_Dplot_old'
Hplot = 'confocal_625ms_40x_Hplot_old'
Hlogplot = 'confocal_625ms_40x_Hlogplot_old'

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.16, 1.62, 1)
conversion[2] = (0.16, 1.62, 1)
conversion[3] = (0.16, 1.62, 1)
# conversion[4] = (0.30, 5.31, 1)
# conversion[5] = (0.30, 5.33, 1)
# conversion[6] = (0.30, 5.33, 1)
# conversion[7] = (0.30, 5.35, 1)

trajectory = dict()

trajectory[1] = np.genfromtxt('./confocal_625ms_40x_1.csv',
            delimiter =",")
trajectory[2] = np.genfromtxt('./confocal_625ms_40x_2.csv',
            delimiter =",")
trajectory[3] = np.genfromtxt('./confocal_625ms_40x_3.csv',
            delimiter =",")
# trajectory[4] = np.genfromtxt('./CON_40x_2D_130ms_4_T.csv',
#             delimiter =",")
# trajectory[5] = np.genfromtxt('./CON_40x_2D_130ms_5_T.csv',
#             delimiter =",")
# trajectory[6] = np.genfromtxt('./CON_40x_2D_130ms_6_T.csv',
#             delimiter =",")
# trajectory[7] = np.genfromtxt('./CON_40x_2D_130ms_7_T.csv',
#             delimiter =",")

totvids = 3

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)

#Get rid of trajectories that are too short, as determined by prettify
final = dict()
tots = dict()

for num in range(1, totvids + 1):
    (final[num], tots[num]) = prettify(trajectory[num], 50, 50, conversion[num][0], conversion[num][1], conversion[num][2])


#Collect trajectories of separate videos into one array
parts = dict()
lehi = dict()

for num in range(1, totvids + 1):
    parts[num] = tots[num]
    lehi[num] = final[num][1]
    counter = 1

    while counter < parts[num]:
        counter = counter + 1
        lehi[num] = np.append(lehi[num], final[num][counter], axis=0)

for num in range(2, totvids + 1):
    lehi[num][:,0] = lehi[num][:,0] + max(lehi[num-1][:, 0])        
        
nephi = lehi[1]
counter = 1

while counter < totvids:
    counter = counter + 1
    nephi = np.append(nephi, lehi[counter], axis=0)

In [3]:
final[1]

{1: array([[  1.00000000e+00,   0.00000000e+00,   2.75756800e+01, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  1.00000000e+00,   1.00000000e+00,   2.75744000e+01, ...,
           1.32710400e-06,   5.04631296e-04,   0.00000000e+00],
        [  1.00000000e+00,   2.00000000e+00,   2.75849600e+01, ...,
           3.48779520e-05,   5.37477120e-05,   0.00000000e+00],
        ..., 
        [  1.00000000e+00,   4.70000000e+01,   2.75720000e+01, ...,
           2.33390298e-07,   2.39518448e-05,   0.00000000e+00],
        [  1.00000000e+00,   4.80000000e+01,   2.75883200e+01, ...,
           2.69611200e-06,   1.09213920e-05,   0.00000000e+00],
        [  1.00000000e+00,   4.90000000e+01,   2.75768000e+01, ...,
           2.07360000e-08,   5.12052245e-08,   0.00000000e+00]]),
 2: array([[  2.00000000e+00,   0.00000000e+00,   2.47961600e+01, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  2.00000000e+00,   1.00000000e+00,   2.47814

In [4]:
D = dict()
SD = dict()
total = dict()
t = dict()

D['pd2D'], SD['pd2D'], total['pd2D'], t['pd2D'] = Dpointder(nephi, 0, 15, 17, 1, '2D')
D['pd1Dx'], SD['pd1Dx'], total['pd1Dx'], t['pd1Dx'] = Dpointder(nephi, 0, 15, 17, 1, '1Dx')
D['pd1Dy'], SD['pd1Dy'], total['pd1Dy'], t['pd1Dy'] = Dpointder(nephi, 0, 15, 17, 1, '1Dy')

print(D['pd2D'], SD['pd2D'])
print(D['pd1Dx'], SD['pd1Dx'])
print(D['pd1Dy'], SD['pd1Dy'])
total['pd1Dx']

0.000401703648 0.00294918847084
0.0004861154496 0.00567112311844
0.0003172918464 0.00114361628627


540

In [5]:
D['linalg2D'], SD['linalg2D'], total['linalg2D'] = Dlinalg(nephi, 0, 15, 9, '2D')
D['linalg1Dx'], SD['linalg1Dx'], total['linalg1Dx'] = Dlinalg(nephi, 0, 15, 9, '1Dx')
D['linalg1Dy'], SD['linalg1Dy'], total['linalg1Dy'] = Dlinalg(nephi, 0, 15, 9, '1Dy')

print(D['linalg2D'], SD['linalg2D'])
print(D['linalg1Dx'], SD['linalg1Dx'])
print(D['linalg1Dy'], SD['linalg1Dy'])

8.53610778805e-05 7.39623741707e-06
8.95370818679e-05 8.55805090456e-06
8.11850738931e-05 8.39709726478e-06


In [6]:
D['linalgw2D'], SD['linalgw2D'], total['linalgw2D'] = Dlinalgw(nephi, 0, 15, 9, '2D')
D['linalgw1Dx'], SD['linalgw1Dx'], total['linalgw1Dx'] = Dlinalgw(nephi, 0, 15, 9, '1Dx')
D['linalgw1Dy'], SD['linalgw1Dy'], total['linalgw1Dy'] = Dlinalgw(nephi, 0, 15, 9, '1Dy')

print(D['linalgw2D'], SD['linalgw2D'])
print(D['linalgw1Dx'], SD['linalgw1Dx'])
print(D['linalgw1Dy'], SD['linalgw1Dy'])

0.000138005837146 8.93230113468e-06
0.000147861448345 1.03325325044e-05
0.000132328575558 8.70019221102e-06


In [19]:
def plot_Mean1DMSDsorDeffLOG(traj, n1, n2, n3, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1
    SD = np.zeros(np.shape(MMSD))
    
    #Now to calculate the standard dev at each point:
    for num in range (1, total1):
        SDunit = (MSD[num] - MMSD)**2
        SD = SD + SDunit
    SD = np.sqrt(SD/total1)
    SE = SD/np.sqrt(total1)
    
    #Linear algebra to find Deff:
    t = time[1][:]
    w = dict()
    line = dict()
    A = np.ones((np.shape(t)[0], 2))
    A[:, 0] = t
    w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
    w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
    w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
    line[0] = w[0][0]*t + w[0][1]
    line[1] = w[1][0]*t + w[1][1]
    line[2] = w[2][0]*t + w[2][1]
    
    #Linear algebra for fit on log plot:
    wl = dict()
    linel = dict()
    lt = np.log(t)
    lA = np.ones((np.shape(t)[0], 2))
    lA[:, 0] = lt
    lM = np.log(MMSD)
    wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
    wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
    wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
    linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
    linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
    linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, MMSD[:, 0], linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, MMSD[:, 1], linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, MMSD[:, 2], linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
    ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
    ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
    ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
    #ax.plot(t, line[0], linewidth=3, color='blue')
    #ax.plot(t, line[1], linewidth=3, color='red')
    #ax.plot(t, line[2], linewidth=3, color='green')
    
    ax.plot(t, linel[0], linewidth=3, color='blue')
    ax.plot(t, linel[1], linewidth=3, color='red')
    ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    plt.yscale('log')
    plt.xscale('log')
    plt.gca().set_xlim([1, limit1])
    plt.gca().set_ylim([0.0001, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return MMSD, total1

In [20]:
one1, total1 = plot_Mean1DMSDsorDeffLOG(nephi, 0, 15, 9, 1, 1, 'MSD (um^2)', logplot, 100.1, 0.1, 1, 1)
plt.show()

In [9]:
def plot_Mean1DMSDsorDeff(traj, n1, n2, n3, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1
    SD = np.zeros(np.shape(MMSD))
    
    #Now to calculate the standard dev at each point:
    for num in range (1, total1):
        SDunit = (MSD[num] - MMSD)**2
        SD = SD + SDunit
    SD = np.sqrt(SD/total1)
    SE = SD/np.sqrt(total1)
    
    #Linear algebra to find Deff:
    t = time[1][:]
    w = dict()
    line = dict()
    A = np.ones((np.shape(t)[0], 2))
    A[:, 0] = t
    w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
    w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
    w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
    line[0] = w[0][0]*t + w[0][1]
    line[1] = w[1][0]*t + w[1][1]
    line[2] = w[2][0]*t + w[2][1]
    
    #Linear algebra for fit on log plot:
    wl = dict()
    linel = dict()
    lt = np.log(t)
    lA = np.ones((np.shape(t)[0], 2))
    lA[:, 0] = lt
    lM = np.log(MMSD)
    wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
    wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
    wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
    linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
    linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
    linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, MMSD[:, 0], linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, MMSD[:, 1], linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, MMSD[:, 2], linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
    ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
    ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
    ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
    ax.plot(t, line[0], linewidth=3, color='blue')
    ax.plot(t, line[1], linewidth=3, color='red')
    ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return MMSD, total1

In [25]:
one1, total1= plot_Mean1DMSDsorDeff(nephi, 0, 15, 9, 0, 2, 'MSD (um^2)', Mplot, 30.1, 0.031, 6, 0.01)
plt.show()

In [11]:
def plot_Mean1DMSDsorDeffold(traj, n1, n2, n3, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(time[1][:], MMSD[:, 0], linewidth=10, label='2D')
    ax.plot(time[1][:], MMSD[:, 1], linewidth=10, label='1D x')
    ax.plot(time[1][:], MMSD[:, 2], linewidth=10, label='1D y')

    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.65, 0.35), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return MMSD

In [29]:
one1 = plot_Mean1DMSDsorDeffold(nephi, 0, 15, 17, 0, 4, 'Deff (um^2/s)', Dplot, 30.1, 0.0021, 6, 0.0004)
plt.show()

In [13]:
one1

array([[  4.01703648e-04,   4.86115450e-04,   3.17291846e-04,
          0.00000000e+00],
       [  7.67549504e-04,   1.34920692e-03,   1.85892083e-04,
          0.00000000e+00],
       [  5.23588834e-04,   8.93644013e-04,   1.53533654e-04,
          0.00000000e+00],
       [  5.19058664e-04,   7.17363118e-04,   3.20754209e-04,
          0.00000000e+00],
       [  4.84832019e-04,   6.30670886e-04,   3.38993152e-04,
          0.00000000e+00],
       [  4.85111051e-04,   5.89831526e-04,   3.80390576e-04,
          0.00000000e+00],
       [  4.11467724e-04,   4.91188584e-04,   3.31746864e-04,
          0.00000000e+00],
       [  3.89641781e-04,   4.67060595e-04,   3.12222967e-04,
          0.00000000e+00],
       [  3.46118905e-04,   4.20319853e-04,   2.71917957e-04,
          0.00000000e+00],
       [  3.52745748e-04,   3.87034738e-04,   3.18456758e-04,
          0.00000000e+00],
       [  3.36955098e-04,   3.64073133e-04,   3.09837062e-04,
          0.00000000e+00],
       [  3.01826879e

In [14]:
def plot_histogram(traj, n1, n2, n3, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])
    
    t = time[1][:]
    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(t, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx, 0]
    
    plot, bins = np.histogram(hist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
    plt.xlabel('D (um2/s)', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [15]:
a, hist, total= plot_histogram(nephi, 0, 15, 17, 'Deff (um^2/s)', Hplot, 1)
plt.show()

In [16]:
def plot_loghistogram(traj, n1, n2, n3, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])
    
    t = time[1][:]
    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(t, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx, 0]
    
    z1, z2 = find_nearest(hist[1:], 0)
    hist = np.delete(hist[1:], z2)
    loghist = np.log(hist)

    plot, bins = np.histogram(loghist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
    plt.xlabel('log(D)', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [17]:
hist, MSD, total = plot_loghistogram(nephi, 0, 15, 17, 'Deff (um^2/s)', Hlogplot, 1)
plt.show()